In [58]:
from __future__ import absolute_import, division, print_function, unicode_literals
!pip install tensorflow-gpu==2.0.0-alpha0
import tensorflow as tf
import numpy as np
import pandas as pd
import os
import datetime
from tensorflow.python.client import device_lib

In [0]:
df = pd.read_csv('peja.csv')
lst = df["Text"].to_numpy().tolist()
text = " ".join(lst)

In [60]:
print("TensorFlow version: ", tf.__version__)
# TODO: import data from scraper

# text = 'placeholder for real song text placeholder for real song text xD'
words = [w for w in text.split(' ') if w.strip() != '' or w == '\n']
print("Text is {} words long".format(len(words)))

vocab = sorted(set(text))
print ('There are {} unique characters'.format(len(vocab)))
char2int = {c:i for i, c in enumerate(vocab)}
int2char = np.array(vocab)
# print('Vector:\n')
# for char,_ in zip(char2int, range(len(vocab))):
#     print(' {:4s}: {:3d},'.format(repr(char), char2int[char]))

text_as_int = np.array([char2int[ch] for ch in text], dtype=np.int32)
print ('{}\n mapped to integers:\n {}'.format(repr(text[:100]), text_as_int[:100]))

TensorFlow version:  2.0.0-alpha0
Text is 136595 words long
There are 114 unique characters
'\nTekst piosenki:\r\n\r\n\t\t    \t\t    \r\n                    A ja robię ten szmal, mam tu muzę, mam tu bal\n'
 mapped to integers:
 [ 1 50 64 70 78 79  3 75 68 74 78 64 73 70 68 26  2  1  2  1  0  0  3  3
  3  3  0  0  3  3  3  3  2  1  3  3  3  3  3  3  3  3  3  3  3  3  3  3
  3  3  3  3  3  3 31  3 69 60  3 77 74 61 68 96  3 79 64 73  3 78 85 72
 60 71 12  3 72 60 72  3 79 80  3 72 80 85 96 12  3 72 60 72  3 79 80  3
 61 60 71  1]


In [61]:
# Create training and validation data from the text
# (make sure the training piece is divisible by the batch size, 64 in this case)
tr_text = text_as_int[:704000] 
val_text = text_as_int[704000:] 
print(text_as_int.shape, tr_text.shape, val_text.shape)

(936423,) (704000,) (232423,)


## Building the model
### Tunable variables

In [0]:
batch_size = 64
buffer_size = 10000
embedding_dim = 256
epochs = 50
seq_length = 200
examples_per_epoch = len(text)//seq_length
#lr = 0.001 #will use default for Adam optimizer
rnn_units = 1024
vocab_size = len(vocab)

### Training an validation datasets

In [63]:
tr_char_dataset = tf.data.Dataset.from_tensor_slices(tr_text)
val_char_dataset = tf.data.Dataset.from_tensor_slices(val_text)
tr_sequences = tr_char_dataset.batch(seq_length+1, drop_remainder=True)
val_sequences = val_char_dataset.batch(seq_length+1, drop_remainder=True)

def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

tr_dataset = tr_sequences.map(split_input_target).shuffle(buffer_size).batch(batch_size, drop_remainder=True)
val_dataset = val_sequences.map(split_input_target).shuffle(buffer_size).batch(batch_size, drop_remainder=True)
print(tr_dataset, val_dataset)

<BatchDataset shapes: ((64, 200), (64, 200)), types: (tf.int32, tf.int32)> <BatchDataset shapes: ((64, 200), (64, 200)), types: (tf.int32, tf.int32)>


### Build the model

In [64]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
     model = tf.keras.Sequential([
     tf.keras.layers.Embedding(vocab_size, embedding_dim,
     batch_input_shape=[batch_size, None]),
     tf.keras.layers.Dropout(0.2),
     tf.keras.layers.LSTM(rnn_units,
     return_sequences=True,
     stateful=True,
     recurrent_initializer='glorot_uniform'),
     tf.keras.layers.Dropout(0.2), 
     tf.keras.layers.LSTM(rnn_units,
     return_sequences=True,
     stateful=True,
     recurrent_initializer='glorot_uniform'),
     tf.keras.layers.Dropout(0.2),
     tf.keras.layers.Dense(vocab_size)
 ])
     return model

model = build_model(
    vocab_size = len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=batch_size)


### Review the output shape and model, and define loss:

In [65]:
model.summary()

for input_example_batch, target_example_batch in tr_dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape)

def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Loss:      ", example_batch_loss.numpy().mean())


Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (64, None, 256)           29184     
_________________________________________________________________
dropout_9 (Dropout)          (64, None, 256)           0         
_________________________________________________________________
unified_lstm_6 (UnifiedLSTM) (64, None, 1024)          5246976   
_________________________________________________________________
dropout_10 (Dropout)         (64, None, 1024)          0         
_________________________________________________________________
unified_lstm_7 (UnifiedLSTM) (64, None, 1024)          8392704   
_________________________________________________________________
dropout_11 (Dropout)         (64, None, 1024)          0         
_________________________________________________________________
dense_3 (Dense)              (64, None, 114)          

In [66]:
optimizer = tf.keras.optimizers.Adam()
model.compile(optimizer=optimizer, loss=loss)
patience = 10
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience)

checkpoint_dir = './checkpoints'+ datetime.datetime.now().strftime("_%Y.%m.%d-%H:%M:%S")
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")
checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)
history = model.fit(tr_dataset, epochs=epochs, callbacks=[checkpoint_callback, early_stop] , validation_data=val_dataset)
print ("Training stopped as there was no improvement after {} epochs".format(patience))

Epoch 1/50
54/54 [==============================] - 26s 487ms/step - loss: 3.4860 - val_loss: 3.2606
Epoch 2/50
54/54 [==============================] - 25s 463ms/step - loss: 3.0241 - val_loss: 2.6716
Epoch 3/50
54/54 [==============================] - 25s 469ms/step - loss: 2.5492 - val_loss: 2.3767
Epoch 4/50
54/54 [==============================] - 25s 472ms/step - loss: 2.3266 - val_loss: 2.1828
Epoch 5/50
54/54 [==============================] - 26s 478ms/step - loss: 2.1415 - val_loss: 2.0195
Epoch 6/50
54/54 [==============================] - 26s 479ms/step - loss: 1.9958 - val_loss: 1.9057
Epoch 7/50
54/54 [==============================] - 26s 485ms/step - loss: 1.8818 - val_loss: 1.8280
Epoch 8/50
54/54 [==============================] - 26s 484ms/step - loss: 1.7903 - val_loss: 1.7686
Epoch 9/50
54/54 [==============================] - 26s 483ms/step - loss: 1.7095 - val_loss: 1.7289
Epoch 10/50
54/54 [==============================] - 26s 483ms/step - loss: 1.6407 - val_lo

### Test the model

In [67]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir)) 
model.build(tf.TensorShape([1, None]))

def generate_text(model, start_string):
    
    print('Generating with seed: "' + start_string + '"')
  
    num_generate = 1000    
    input_eval = [char2int[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)
    text_generated = []
    temperature = 1.0
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions,      num_samples=1)[-1,0].numpy()
        input_eval = tf.expand_dims([predicted_id], 0)
        text_generated.append(int2char[predicted_id])    
    return (start_string + ''.join(text_generated))

print(generate_text(model, start_string="Głucha noc"))

Generating with seed: "Głucha noc"
Głucha noce i dla tego
Te kocie ratusze słowa koncert - odważnych
recytach te sprzedaż ten to wszystko
Warszawia jest w nas, mój gupi się przegaz
Wciąż z fawałkami skrudnie i w górze czasu nagrania 
Wciąż paęarę w to, że kocham i i tam bez bogna?
I rozgrawam cały krzachu i Co Co ty jest na drzeżwa
Ry jesteś z nami wraz ze słaby. Bólk po prostu i gdzie ma kobiet
Jękresz, będzie wiem gdzie tego nie powiem
Muże miasto znany scenicznym nę kocham Rycha dotacz
Opranalenciściem tylko które nazwać sie mówki na sukcy
		    Jarzej - ja rozpustę, i dla wszystkich ludzi 
rap na blęski zachęcy nabruczałem ja szamste
Wieczorem waga wodzę i gówno świat się na wodzie

Ja jestem grupi do miniu, od to, nie wierzyłem ciebie 
Wiel o co chodzi, To mój lat?, o kto solodne gadle
Odczuwaj to próbował - czy to się nie dałem 
odszelu mówisz na bicie bo on to Bysi znak 

Boistwo sławo i głoszyków często piszę się w tych sercach
Gdy mi chuji to nie blefa i i tu na niby do przodo

## Save the model

In [74]:
# print(generate_text(model, start_string="Tede to mój ziom"))

model.save('./pejaModel.h5')

Generating with seed: "Tede to mój ziom"
Tede to mój ziomek

Tylko w życiu mocysz głowę mago. Bierz dobra pęcha
Zawsze kam się bardzo poniszczoną kimś będzie teś tak dobrze
byłości patrzeć na tym to coś bę nie bała
Jestem tylko życie schodzę na pieprzone człowieka
Jesteś swoją płatkę w zamieniach ulicznych wojanów,
Mamy zarybka krótkiem siedzieć ciągle jeźyce
Ale czy się skończyć? poważny na coś zawsze
Jestem zwykłym skurwielami tej refytycji, czujesz hip-hop'i komercja
Refrezentuję kobietę, spokój ogradę czarną pół
W dzisiaj grzieści kręcę, chcę wyrażę się te wszystkie mijasto
Dlatego ścianym wlacy za życiam na pewno świata
Byst ze mną bezsanności z plastą fozorcjy i skrotnie
Wyzwa na 54 Manko Qodro FrLWklaJ 
Czy chujowe chwila, gdy stariałeś się zachwycę
Nie mieć czasu uderzyć na zawsze zmysłonować 
chcę tego rzeszczego sna granku, zapierdalam na rozlebani 
Żyjęc razem obiecznego promu wzrok mi ziomków recenzkę
Ten chatnie pyska
Teraz powiem przykry droga się prodziwa 
więc będzie ta